# TSUMUGIに必要なアノテーション情報を整理する

In [1]:
# Move up to top directory
import os
from pathlib import Path

print(os.getcwd())

while not Path("LICENSE").exists():
    os.chdir('../')

print(os.getcwd())

/mnt/e/Research/TSUMUGI/notebooks/notebools-web
/mnt/e/Research/TSUMUGI


In [2]:
from pathlib import Path
from collections import defaultdict
from itertools import combinations
import csv
import pandas as pd
from matplotlib import pyplot as plt
import pickle
import json

In [3]:
# def write_jsonl(data_dict: dict[list[str]], file_path: str | Path):
#     # JSON Lines形式で保存
#     with open(file_path, "w", encoding="utf-8") as f:
#         for key, values in data_dict.items():
#             json_line = {key: values}
#             f.write(json.dumps(json_line) + "\n")

In [4]:
# def read_jsonl(file_path: str | Path) -> dict[list[str]]:
#     # JSON Lines形式で読み込み
#     data_dict = defaultdict(list)
#     with open(file_path, "r", encoding="utf-8") as f:
#         for line in f:
#             entry = json.loads(line)
#             for key, values in entry.items():
#                 data_dict[key].extend(values)
#     return dict(data_dict)


In [5]:
path_data = Path("data", "statistical_filtered.csv")

In [17]:
data = pd.read_csv(path_data)
print(len(data))

46991


/tmp/ipykernel_407517/4252504432.py:1: DtypeWarning: Columns (25,60) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(path_data)


In [31]:
print(data.columns)

Index(['phenotyping_center', 'intercept_estimate', 'procedure_id',
       'mutant_biological_model_id', 'rotated_residuals_test',
       'weight_effect_p_value', 'male_mutant_count', 'pipeline_stable_key',
       'female_ko_effect_p_value', 'pipeline_stable_id',
       'parameter_stable_key', 'data_type', 'parameter_stable_id',
       'interaction_significant', 'strain_accession_id',
       'control_selection_method', 'parameter_name', 'allele_name',
       'phenotyping_center_id', 'weight_effect_stderr_estimate',
       'weight_effect_parameter_estimate', 'procedure_stable_id', 'status',
       'sex_effect_parameter_estimate', 'female_ko_effect_stderr_estimate',
       'female_percentage_change', 'group_2_residuals_normality_test',
       'marker_accession_id', 'mp_term_name',
       'group_1_residuals_normality_test', 'genotype_effect_p_value',
       'dependent_variable', 'resource_name', 'project_id', 'procedure_name',
       'doc_id', 'top_level_mp_term_id', 'allele_accession_id',

In [7]:
Path("data/annotation").mkdir(exist_ok=True, parents=True)

In [32]:
import numpy as np

threshold = 0.0001

# 条件リスト
conditions = [
    (data["sex_effect_p_value"] < threshold) & (data["female_ko_effect_p_value"] < threshold) & (data["male_ko_effect_p_value"] > threshold),
    (data["sex_effect_p_value"] < threshold) & (data["male_ko_effect_p_value"] < threshold) & (data["female_ko_effect_p_value"] > threshold)
]

# 条件に対応する値
choices = ["♀", "♂"]

# np.selectで列を設定
data["sexdual_dimorphism"] = np.select(conditions, choices, default=None)

# 結果を確認
print(data["sexdual_dimorphism"].value_counts())


sexdual_dimorphism
♂    4864
♀    4137
Name: count, dtype: int64


In [38]:
data[["p_value", "genotype_effect_p_value", "sex_effect_p_value", "sexdual_dimorphism", "female_ko_effect_p_value", "male_ko_effect_p_value", "female_percentage_change", "male_percentage_change"]]

,p_value,genotype_effect_p_value,sex_effect_p_value,sexdual_dimorphism,female_ko_effect_p_value,male_ko_effect_p_value,female_percentage_change,male_percentage_change
0,4.054573e-01,4.054573e-01,8.131008e-08,♂,4.054573e-01,6.088797e-06,-20.5266253870529,-120.408713480008
1,0.000000e+00,NaN,NaN,None,NaN,NaN,NaN,NaN
2,0.000000e+00,NaN,NaN,None,NaN,NaN,NaN,NaN
3,1.144336e-06,1.144336e-06,0.000000e+00,♀,1.144336e-06,2.957211e-02,211.171790862029,-106.457814174796
4,1.273780e-06,1.273780e-06,NaN,None,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
46986,8.729624e-06,NaN,NaN,None,2.502193e-04,1.473660e-02,NaN,NaN
46987,1.578099e-07,1.578099e-07,1.016244e-01,None,8.359806e-04,3.485404e-06,-112.230615708817,-187.950048394387
46988,5.775577e-01,5.775577e-01,0.000000e+00,♂,5.735144e-01,5.518938e-07,-25.5958136721999,-159.145484509104
46989,2.756679e-19,NaN,NaN,None,7.356204e-15,2.588011e-06,NaN,NaN


## marker_symbol と mp_term_name の対応付け

In [39]:
data_select = data[['marker_symbol', 'mp_term_name', 'zygosity', "sexdual_dimorphism"]].drop_duplicates()
# data_select = data[['marker_symbol', 'marker_accession_id', 'mp_term_name', 'mp_term_id']].drop_duplicates()
data_select

,marker_symbol,mp_term_name,zygosity,sexdual_dimorphism
0,Rhd,decreased circulating HDL cholesterol level,homozygote,♂
1,Dpf2,"preweaning lethality, complete penetrance",homozygote,None
2,Gna13,"preweaning lethality, complete penetrance",homozygote,None
3,Dynlrb2,increased fasting circulating glucose level,heterozygote,♀
4,Slc16a2,increased circulating calcium level,homozygote,None
...,...,...,...,...
46982,Arhgap21,abnormal auditory brainstem response,heterozygote,None
46984,Prkdc,increased monocyte cell number,homozygote,None
46985,Cfap20dc,decreased heart weight,homozygote,♂
46988,Prmt7,decreased hematocrit,homozygote,♂


In [92]:
data_select["zygosity"].value_counts()

zygosity
homozygote      28641
heterozygote     8935
hemizygote        465
Name: count, dtype: int64

In [45]:
data_dict = defaultdict(list)
for index, row in data_select.iterrows():
    if row['zygosity'] == 'homozygote':
        annotate = "Homo"
    elif row['zygosity'] == 'heterozygote':
        annotate = "Hetero"
    else:
        annotate = "Hemi"

    if row['sexdual_dimorphism'] == "♀":
        annotate = ", ".join([annotate, '♀'])
    if row['sexdual_dimorphism'] == "♂":
        annotate = ", ".join([annotate, '♂'])

    data_dict[row['marker_symbol']].append(f"{row['mp_term_name']} ({annotate})")   

for key in data_dict.keys():
    data_dict[key].sort()

print(data_dict["Rhd"])

['abnormal skin condition (Homo)', 'decreased circulating HDL cholesterol level (Homo, ♂)', 'decreased circulating alkaline phosphatase level (Homo, ♀)', 'decreased circulating cholesterol level (Homo, ♂)', 'decreased circulating free fatty acids level (Homo)', 'decreased hemoglobin content (Homo, ♂)', 'decreased mean corpuscular hemoglobin (Homo)', 'decreased mean corpuscular hemoglobin concentration (Homo)', 'decreased mean corpuscular volume (Homo)', 'increased exploration in new environment (Homo)']


In [46]:
file_path = "data/annotation/symbol_mptermname.json"
json.dump(data_dict, open(file_path, "w"), indent=4, sort_keys=True)


In [47]:
%%bash

head -n 10 data/annotation/symbol_mptermname.json

{
    "0610010K14Rik": [
        "embryonic lethality prior to organogenesis (Homo)",
        "embryonic lethality prior to tooth bud stage (Homo)",
        "increased startle reflex (Hetero, \u2642)",
        "preweaning lethality, complete penetrance (Homo)"
    ],
    "0610040J01Rik": [
        "abnormal heart morphology (Homo)",
        "abnormal spleen morphology (Homo)",


## Connect mp_term_name to IMPC Phenotype URL

In [75]:
data_select = data[['mp_term_id', 'mp_term_name']].drop_duplicates()
# data_select = data[['marker_symbol', 'marker_accession_id', 'mp_term_name', 'mp_term_id']].drop_duplicates()
data_select

,mp_term_id,mp_term_name
0,MP:0000186,decreased circulating HDL cholesterol level
1,MP:0011100,"preweaning lethality, complete penetrance"
3,MP:0013279,increased fasting circulating glucose level
4,MP:0000194,increased circulating calcium level
5,MP:0005011,increased eosinophil cell number
...,...,...
44618,MP:0001562,abnormal circulating calcium level
46141,MP:0003655,absent pancreas
46159,MP:0005498,hyporesponsive to tactile stimuli
46562,MP:0008206,increased B-2 B cell number


In [76]:
data_dict_url = dict()
for index, row in data_select.iterrows():
    mp_tern_id = row['mp_term_id']
    impc_url = f"https://www.mousephenotype.org/data/phenotypes/{mp_tern_id}"
    mp_term_name = row['mp_term_name']
    data_dict_url[mp_term_name] = impc_url

print(data_dict_url["small lymph nodes"])

https://www.mousephenotype.org/data/phenotypes/MP:0002217


In [77]:
with open('data/annotation/mptermname_phenotypeurl.tsv', 'w') as f:
    for term, url in data_dict_url.items():
        f.write(f"{term}\t{url}\n")

In [78]:
%%bash

head -n 3 data/annotation/mptermname_phenotypeurl.tsv

decreased circulating HDL cholesterol level	https://www.mousephenotype.org/data/phenotypes/MP:0000186
preweaning lethality, complete penetrance	https://www.mousephenotype.org/data/phenotypes/MP:0011100
increased fasting circulating glucose level	https://www.mousephenotype.org/data/phenotypes/MP:0013279


## Connect marker_symbol to accession_id

In [79]:
data_select = data[['marker_symbol', 'marker_accession_id']].drop_duplicates()
# data_select = data[['marker_symbol', 'marker_accession_id', 'mp_term_name', 'mp_term_id']].drop_duplicates()
data_select

,marker_symbol,marker_accession_id
0,Rhd,MGI:1202882
1,Dpf2,MGI:109529
2,Gna13,MGI:95768
3,Dynlrb2,MGI:1922715
4,Slc16a2,MGI:1203732
...,...,...
46753,Amigo3,MGI:2444854
46806,Sec1,MGI:1928893
46833,Slc23a3,MGI:104516
46894,Agtr2,MGI:87966


In [80]:
data_dict = dict()
for index, row in data_select.iterrows():
    data_dict[row['marker_symbol']] = row['marker_accession_id']

In [81]:
data_dict["Ncam1"]

'MGI:97281'

In [82]:
json.dump(data_dict, open("data/annotation/symbol_mgiid.json", "w"), indent=4, sort_keys=True)
Path("data/annotation/symbol_mgiid.tsv").write_text("\n".join([f"{k}\t{v}" for k, v in data_dict.items()]))

137963

In [83]:
%%bash
head -n 3 data/annotation/symbol_mgiid.json
head -n 3 data/annotation/symbol_mgiid.tsv

{
    "0610010K14Rik": "MGI:1915609",
    "0610040J01Rik": "MGI:1923511",
Rhd	MGI:1202882
Dpf2	MGI:109529
Gna13	MGI:95768
